In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/qaly_input_full.csv")

In [3]:
to_fit

,Description,Value,Lower Bound,Upper Bound
0,Child-Ototoxicity,25.66,23.80,27.51
1,Child-NCD,20.71,17.31,22.87


In [4]:
# fits an encompassing beta distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_beta_distr(value, lower_bound, upper_bound):
    og_alpha = value
    og_beta = 1-og_alpha
    
    
    
    for scale in range(2000):
        scale = 2000 - scale
        alpha = og_alpha * scale
        beta = og_beta * scale
        ppf_025 = ss.beta.ppf(0.025, alpha, beta)
        ppf_975 = ss.beta.ppf(0.975, alpha, beta)
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            print("alpha: %.2f \t beta: %.2f" % (alpha, beta))           
            return alpha, beta
        
    scale_lower_bound = 0.01
    scale_upper_bound = 0.5
    linspace_points = 0.01
    search_range = np.arange(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    print("No match found")
    return 0, 0

In [5]:
scale_lower_bound = 0.01
scale_upper_bound = 0.5
linspace_points = 0.01
search_range = np.arange(scale_lower_bound, scale_upper_bound, linspace_points)


In [6]:
search_range

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49])

In [7]:
to_fit["alpha"], to_fit["beta"] = np.vectorize(fit_encompassing_beta_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

a: 641.50 	 scale: 0.04
a: 641.50 	 scale: 0.04
a: 129.44 	 scale: 0.16


In [8]:
to_fit["beta_mean"] = ss.gamma.mean(to_fit["alpha"], to_fit["beta"])
to_fit["beta_2.5pct"] = ss.gamma.ppf(0.025, a=to_fit["alpha"], scale=to_fit["beta"])
to_fit["beta_97.5pct"] = ss.gamma.ppf(0.975, a=to_fit["alpha"], scale=to_fit["beta"])

In [9]:
to_fit

,Description,Value,Lower Bound,Upper Bound,alpha,beta,beta_mean,beta_2.5pct,beta_97.5pct
0,Child-Ototoxicity,25.66,23.80,27.51,641.5000,0.04,641.5400,23.712486,27.683277
1,Child-NCD,20.71,17.31,22.87,129.4375,0.16,129.5975,17.296094,24.426826


In [10]:
to_fit.to_csv("Data/qaly_output_full.csv")